# Projet sante publique france

## Exploratoration initiale

Les données sont accessibles sous forme de `CSV` (**Le fichier CSV est assez conséquent env. 2GB.**) ou éventuellement si la base devient trop lourde pour être chargée en RAM d'un bloc, les données peuvent être restaurées dans une base de données MongoDB. 

Pour charger les données, un script est disponible dans le répertoire /src. Le plus simple est de créer un environement virtuel, installer les dépendances et exécuter make data. 


## Nettoyage des données 

Les données étant crées par les utilisateurs, les données sont "sales". Il faut retirer toutes les données qui sont des erreurs de saisie.
   * Données provenant de sources fiables vs. sources non fiables.
       * Certaines données sont entrées par des organismes "fiables" exemple carrefour, fleury-michon...
   * Retirer les produits n'ayant pas de nom (product_name == nan).
   * Sauvegarde des données dans un fichier intermédiaire


Le nettoyage des données sera traité plus en détails dans le notebook suivant. Les données doivent être nettoyées de manière statistique.

### Chargement des données

NB: Les données sont assez lourdes, le chargement peut prendre un certain temps (en fonction de l'ordinateur utilisé)

In [1]:
from IPython.display import display
import pandas as pd

In [2]:
%%time
data = pd.read_csv('../data/raw/products.csv', delimiter='\t', low_memory=False)

CPU times: user 28.4 s, sys: 4.19 s, total: 32.6 s
Wall time: 32.6 s


On ne récupère que les données concernant la France et les pays voisins francophones.

In [3]:
data = data[(data['countries_tags'] == 'en:france') | 
            (data['countries_tags'] == 'en:belgium') |
            (data['countries_tags'] == 'en:switzerland')|
            (data['countries_tags'] == 'en:luxembourg')]

In [4]:
data.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000100,http://world-en.openfoodfacts.org/product/0000...,del51,1444572561,2015-10-11T14:09:21Z,1444659212,2015-10-12T14:13:32Z,moutarde au moût de raisin,NaN,100g,...,NaN,18.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00000000001111111111,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1560020173,2019-06-08T18:56:13Z,1560020173,2019-06-08T18:56:13Z,Sfiudwx,NaN,dgesc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0000000000123,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1535737982,2018-08-31T17:53:02Z,1535737986,2018-08-31T17:53:06Z,Sauce Sweety chili 0%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


On peut regarder les contributeurs ainsi que le nombre de contribution qu'ils ont fourni

In [5]:
with pd.option_context("display.max_rows", 30):
    display(data['creator'].value_counts().head(30))

kiliweb                       464711
openfoodfacts-contributors     70378
date-limite-app                 9777
openfood-ch-import              8604
tacite                          7834
sebleouf                        6974
foodrepo                        3225
teolemon                        2425
tacinte                         2159
stephane                        1931
agamitsudo                      1821
jeanbono                        1584
segundo                         1416
miles67off                      1337
phoenix                          910
aleene                           793
jacob80                          651
beniben                          588
nouky20                          505
scamark                          500
moon-rabbit                      494
andre                            474
cestki13                         459
systeme-u                        439
big-brother                      426
foodvisor                        418
hommedanslalune                  415
c

On peut commencer par définir des sources "fiables". En effet, certains producteurs renseignent eux-même les données (les données sont de bonnes qualité)

In [6]:
TRUSTED_SOURCES = ['casino', 'carrefour', 'ldc', 'fleury-michon', 'ferrero', 'systeme-u', 'barilla']

On drop les produits pour lesquels on a pas de nom.

In [7]:
print(f'Nombre de lignes avant supression {data.shape[0]}')
data.drop(data[data['product_name'].isna()].index, axis=0, inplace=True)
print(f'Nombre de lignes après supression {data.shape[0]}')

Nombre de lignes avant supression 619678
Nombre de lignes après supression 600654


In [8]:
data['creator'].value_counts()

kiliweb                       462956
openfoodfacts-contributors     57214
date-limite-app                 9067
openfood-ch-import              8252
tacite                          7821
                               ...  
lmbrt                              1
tomadeb                            1
gscout                             1
lorel-x                            1
nc29                               1
Name: creator, Length: 5291, dtype: int64

In [9]:
data['trusted_src'] = data['creator'].map(lambda x : True if x in TRUSTED_SOURCES else False)

In [10]:
data

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,trusted_src
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,0000000000100,http://world-en.openfoodfacts.org/product/0000...,del51,1444572561,2015-10-11T14:09:21Z,1444659212,2015-10-12T14:13:32Z,moutarde au moût de raisin,NaN,100g,...,18.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,00000000001111111111,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1560020173,2019-06-08T18:56:13Z,1560020173,2019-06-08T18:56:13Z,Sfiudwx,NaN,dgesc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5,0000000000123,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1535737982,2018-08-31T17:53:02Z,1535737986,2018-08-31T17:53:06Z,Sauce Sweety chili 0%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054795,99999971,http://world-en.openfoodfacts.org/product/9999...,openfoodfacts-contributors,1546519940,2019-01-03T12:52:20Z,1555593435,2019-04-18T13:17:15Z,test,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1054796,99999988,http://world-en.openfoodfacts.org/product/9999...,teolemon,1546519833,2019-01-03T12:50:33Z,1563296976,2019-07-16T17:09:36Z,Boules,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1054797,9999999004360,http://world-en.openfoodfacts.org/product/9999...,kiliweb,1548086277,2019-01-21T15:57:57Z,1558357406,2019-05-20T13:03:26Z,Minis beignets,Beignets natures sucrés,NaN,...,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1054798,99999995,http://world-en.openfoodfacts.org/product/9999...,kiliweb,1538818432,2018-10-06T09:33:52Z,1552167501,2019-03-09T21:38:21Z,Steak haché pur boeuf,NaN,NaN,...,-2.0,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


Sauvegarde des données partiellement nettoyées

In [11]:
# to_pickle run faster than to_csv
data.to_pickle('../data/interim/products_interim.pickle')